### 1. 미세먼지(`한국환경공단_에어코리아_대기오염정보`)관련 API를 활용하여 다음 질문에 답하시오.
    a. 시도별 실시간 측정정보 조회에서  확인가능한 시도 이름을 전부 작성하시오.
        -서울인지 서울특별시인지 등 parameter를 나열하세요
*전국, 서울, 부산, 대구, 인천, 광주, 대전, 울산, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주, 세종) 이런식으로 QueryParam*

    2. 시도별 실시간 측정정보 조회의 서울의 데이터에 대해, 초 미세먼지 농도가 가장 낮은 stationName을 찾으시오.

        b.시도별 실시간 측정정보 조회의 제주의 데이터에 대해, 초 미세먼지 농도가 가장 낮은 stationName을 찾으시오.
    3. 시도별 실시간 측정정보 조회의 서울의 데이터를 stationName으로 접근하기 쉬운 자료구조로 재구성하시오. 

    4. 종로구의 미세먼지 농도, 초미세먼지 농도의 1달치 데이터를 정리하시오.

In [ ]:
# 예시 URL - https://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?serviceKey=18d15feeaf24d8bd6d59c4cccf127fad660c985a967549f67d2791afc661c73d&returnType=json&numOfRows=100&pageNo=1&sidoName=%EC%84%9C%EC%9A%B8&ver=1.0
BASE_URL = "https://apis.data.go.kr/B552584/ArpltnInforInqireSvc"
from pprint import pprint
import requests
import os
from dotenv import load_dotenv

# 미세먼지
load_dotenv()
public_api_key = os.getenv('PUBLIC_DATA_KEY')
API_KEY = public_api_key

import requests
from urllib.parse import urljoin

def public_api_get(path: str, *, api_key: str = API_KEY, timeout: int = 15, **params):

    path = path.lstrip("/")
    url = f"{BASE_URL}/{path}"

    req_params = {"serviceKey": api_key, **params}

    res = requests.get(url, params=req_params, timeout=timeout)
    res.raise_for_status()

    if req_params.get("returnType", "").lower() == "json":
        try:
            return res.json()
        except ValueError:
            return res.text
    return res.text


response = public_api_get(
    "/getCtprvnRltmMesureDnsty",
    returnType="json",
    numOfRows=100,
    pageNo=1,
    sidoName="서울",
    ver="1.2",
)  

api_response_data = response.get('response')
response_data = api_response_data["body"]["items"]

# pprint(response_data)

new_dict = {}
pm_25_lst = []
new_lst = []
min_pm25_value = float("inf")

for data in response_data:
    pm10 = data.get('pm10Value')
    pm25_flag = data.get('pm25Flag')
    pm10_flag = data.get('pm10Flag')
    pm25 = data.get('pm25Value')
    stationName = data.get('stationName')
    min_stationName = None

    if pm10_flag == '통신장애' or pm25_flag == '통신장애' :
        continue
    
    pm_25_int = int(pm25)
    pm_10_int = int(pm10)
    
    new_dict = {
        "측정소" : stationName,
        "미세먼지" : pm_10_int,
        "초미세먼지" : pm_25_int,
        "통신 상태" : pm25_flag
    }
    
    new_lst.append(new_dict)
    
    
min_item = min(new_lst, key=lambda x: x["초미세먼지"])
min_value = min_item["초미세먼지"]
min_stationName = min_item["측정소"]

pprint(min_item)    
# pprint(min_value)   
# pprint(min_stationName)    



In [ ]:
BASE_URL = "https://apis.data.go.kr/B552584/ArpltnInforInqireSvc"
from pprint import pprint
import requests
import os
from dotenv import load_dotenv

# 미세먼지
load_dotenv()
public_api_key = os.getenv('PUBLIC_DATA_KEY')
API_KEY = public_api_key

import requests

def public_api_get(path: str, *, api_key: str = API_KEY, timeout: int = 15, **params):

    path = path.lstrip("/")
    url = f"{BASE_URL}/{path}"

    req_params = {"serviceKey": api_key, **params}

    res = requests.get(url, params=req_params, timeout=timeout)
    res.raise_for_status()

    if req_params.get("returnType", "").lower() == "json":
        try:
            return res.json()
        except ValueError:
            return res.text
    return res.text


response = public_api_get(
    "/getMsrstnAcctoRltmMesureDnsty",
    returnType="json",
    numOfRows=1000,
    dataTerm="MONTH",
    pageNo=1,
    stationName="종로구",
    ver="1.0",
)  

api_response_data = response.get('response')
response_data = api_response_data["body"]["items"]

pprint(len(response_data))
pprint(response_data[-1]['dataTime'])
pprint(response_data[0]['dataTime'])


